In [1]:
import glob
import os
import subprocess
import tarfile
import wget

import random
import numpy as np
import pandas as pd
import torch

from sklearn.model_selection import train_test_split

import nemo
import nemo.collections.asr as nemo_asr

import json
import librosa

import librosa
import json
import copy

from omegaconf import DictConfig
import copy

import pytorch_lightning as pl
from IPython.utils import io
from datetime import datetime
import json

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

from itertools import zip_longest
from typing import Any, Callable, Dict, List, Optional

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.utilities import rank_zero_info

import json
from nemo.collections.tts.models import MelGanModel
from nemo.collections.tts.models import MixerTTSModel
import soundfile as sf

from nemo.utils.exp_manager import exp_manager
import random

from tqdm import tqdm_notebook

/media/boris/F/anaconda3/envs/nemo/lib/python3.9/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)
################################################################################
### WARNING, path does not exist: KALDI_ROOT=media/boris/F/pykaldi/kaldi
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-05-26 18:31:48 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [2]:
import os
from omegaconf import DictConfig

try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML

yaml = YAML(typ='safe')

In [3]:
# from parts.models import EncDecCTCModelBPE_extended

In [4]:
root_dir = '/media/boris/F/Research_final/'

In [5]:
# model = EncDecCTCModelBPE_extended.from_config_dict(DictConfig(asr_cfg['model']))

In [6]:
def flip_batch(batch):
    new_batch = torch.clone(batch[0]), torch.clone(batch[1]), torch.clone(batch[2]), \
        torch.clone(batch[3])
    for i in range(len(new_batch)):
        try:
            for j in range(len(new_batch[i])):
                new_batch[i][j] = torch.flip(new_batch[i][j], dims=[0])
        except Exception:
            pass
    return new_batch

In [7]:
class MixerTTSModel_extended(MixerTTSModel):
    def __init__(self, cfg: DictConfig, trainer=None, corruption_prob=0, use_dae=False, use_bsm=False):
        super().__init__(cfg=cfg, trainer=trainer)
        self.corruption_prob = corruption_prob
        
        self.use_dae = use_dae
        self.use_bsm = use_bsm
    
    
    def forward(self, text, text_len, pitch=None, spect=None, spect_len=None, attn_prior=None, lm_tokens=None):
        
        if self.use_dae:
            for i in range(text.shape[0]):
                if i // 2:
                    for j in range(text.shape[1]):
                        if random.random() < self.corruption_prob:
                            text[i][j] = random.randint(0, 339)
        
        return super().forward(text=text,
                        text_len=text_len,
                        pitch=pitch,
                        spect=spect,
                        spect_len=spect_len,
                        attn_prior=attn_prior,
                        lm_tokens=lm_tokens)
        

In [8]:
class SpeechChainModel(nemo_asr.models.EncDecCTCModelBPE):
    def __init__(self, cfg: DictConfig, trainer=None, corruption_prob=0):
        super().__init__(cfg=cfg, trainer=trainer)
        
        self.use_dae = cfg['speech_chain'].get('use_dae', False)
        self.corruption_prob = cfg['speech_chain'].get('corruption_prob', 0)
        self.use_bsm = cfg['speech_chain'].get('use_bsm', False)
        
        spec_gen_config_path = cfg['speech_chain'].get('spec_gen_config_path', None)
        with open(spec_gen_config_path) as f:
            spec_gen_cfg = yaml.load(f)
        
        self.spec_gen = MixerTTSModel.from_config_dict(DictConfig(spec_gen_cfg['model']))
        
#         self.spec_gen.use_dae = True
#         self.spec_gen.use_bsm = True
#         self.spec_gen.corruption_prob = 0.5
        
        self.vocoder = MelGanModel.from_pretrained(model_name="tts_melgan").cuda()
       
        self.last_epoch_asr = 0
        self.last_epoch_spec_gen = 0
        self.epochs_to_pretrain = cfg['speech_chain'].get('epochs_to_pretrain', 10)
        
        self.paired_manifest_path = cfg['speech_chain'].get('paired_manifest_path', None)
        self.unpaired_text_manifest_path = cfg['speech_chain'].get('unpaired_text_manifest_path', None)
        self.unpaired_speech_manifest_path = cfg['speech_chain'].get('unpaired_speech_manifest_path', None)
    
        os.system('mkdir tmp')
    
    def training_step(self, batch, batch_idx):
    
        ### Biderectional sequence modeling
        if self.use_bsm:
            flipped_batch = flip_batch(batch)

            new_batch = []

            new_batch.append(torch.vstack((batch[0], flipped_batch[0])))
            new_batch.append(torch.hstack((batch[1], flipped_batch[1])))
            new_batch.append(torch.vstack((batch[2], flipped_batch[2])))
            new_batch.append(torch.hstack((batch[3], flipped_batch[3])))
            batch = new_batch
        
        ###
        
        return super().training_step(batch, batch_idx)
    
    def forward(
        self, input_signal=None, input_signal_length=None, processed_signal=None, processed_signal_length=None
    ):
        
        processed_signal, processed_signal_length = self.preprocessor(
            input_signal=input_signal, length=input_signal_length,
        )
        
        ### Denoising auto-encoder
        if self.use_dae:
            for i in range(processed_signal.shape[0]):
                if i // 2:
                    corruption = np.random.choice([0,1], (processed_signal[i].shape[0], processed_signal[i].shape[1]),
                                                  p=[self.corruption_prob, 1 - self.corruption_prob])
                    processed_signal[i] = processed_signal[i].cpu() * corruption
                    processed_signal[i].cuda()

            ###
        
        encoded, encoded_len = self.encoder(audio_signal=processed_signal, length=processed_signal_length)
        log_probs = self.decoder(encoder_output=encoded)
        greedy_predictions = log_probs.argmax(dim=-1, keepdim=False)

        return log_probs, encoded_len, greedy_predictions
    
    def predict_audio(self):
        os.system(f'rm ./tmp/train_manifest_asr.json')
        os.system(f'cp {self.paired_manifest_path} ./tmp/train_manifest_asr.json')
        
        with open(self.unpaired_text_manifest_path) as val:
              with open('./tmp/train_manifest_asr.json', 'a') as train:           
                    for line in val.readlines():
                        try:
                            sample = json.loads(line)
                            text = sample['text']

                            parsed = self.spec_gen.parse(text)
                            spectrogram = self.spec_gen.cuda().generate_spectrogram(tokens=parsed.cuda())
                            audio = self.vocoder.convert_spectrogram_to_audio(spec=spectrogram)

                            sf.write(f"generated_audio/{text}.wav", audio[0].to('cpu').detach().numpy(), 22050)

                            sample['audio_filepath'] = f"generated_audio/{text}.wav"
                            sample['duration'] = np.round(len(audio[0]) / 22050, 1)

                            train.write(str(json.dumps(sample)) + '\n')

                            del sample, text, parsed, spectrogram, audio
                        except:
                            continue
        
        d = dict(self.cfg)
        d['train_ds']['manifest_filepath'] = './tmp/train_manifest_asr.json'
        self.setup_training_data(DictConfig(d['train_ds']))
    
    def predict_text(self):
        os.system(f'rm ./tmp/train_manifest_spec_gen.json')
        os.system(f'cp {self.paired_manifest_path} ./tmp/train_manifest_spec_gen.json')
        
        with open(self.unpaired_speech_manifest_path) as val:
              with open('./tmp/train_manifest_spec_gen.json', 'a') as train:           
                    for i, line in enumerate(val.readlines()):
                        sample = json.loads(line)
                        text = self.transcribe([sample['audio_filepath']])[0].lstrip().rstrip()
                        if text == "" or ('\u2047' in text):
                            del sample, text
                            continue
                            
                        sample['text'] = text
                    
                        train.write(str(json.dumps(sample)) + '\n')
                    
                        del sample, text
        
        d = dict(self.spec_gen.cfg)
        d['train_ds']['dataset']['manifest_filepath'] = './tmp/train_manifest_spec_gen.json'
        
        os.system('rm -r /media/boris/F/Research_final/data/mixer_tts_sup_data_folder')
        os.system('mkdir /media/boris/F/Research_final/data/mixer_tts_sup_data_folder')
        
        self.spec_gen.setup_training_data(DictConfig(d['train_ds']))
        
    def train_spec_gen(self):
    
        if self.last_epoch_spec_gen < self.epochs_to_pretrain:
            
            checkpoint_callback = pl.callbacks.ModelCheckpoint(
                dirpath='./spec_gen_saves',
                filename='just_last_' + str(self.epochs_to_pretrain),
                save_top_k=-1
            )
            
            trainer_sp = pl.Trainer(devices=1, accelerator='gpu', max_epochs=self.epochs_to_pretrain,
                                 check_val_every_n_epoch=self.epochs_to_pretrain,
                                 num_sanity_val_steps=0,
                                 callbacks=[checkpoint_callback],
#                                  precision=16,
                                 accumulate_grad_batches=1,
                                 gradient_clip_val=1000)

            self.last_epoch_spec_gen = self.epochs_to_pretrain
        
        else:
            checkpoint_callback = pl.callbacks.ModelCheckpoint(
                dirpath='./spec_gen_saves',
                filename='just_last_' + str(self.last_epoch_spec_gen),
                save_top_k=-1
            )
            
            trainer_sp = pl.Trainer(devices=1, accelerator='gpu', max_epochs=self.last_epoch_spec_gen,
                                 check_val_every_n_epoch=1,
                                 num_sanity_val_steps=0,
                                 callbacks=[checkpoint_callback],
                                 resume_from_checkpoint=f'spec_gen_saves/just_last_{self.last_epoch_spec_gen - 1}.ckpt',
#                                  precision=16,
                                 accumulate_grad_batches=1,
                                 gradient_clip_val=1000)
        
        self.spec_gen.set_trainer(trainer_sp)
        trainer_sp.fit(self.spec_gen)
            
        if self.last_epoch_spec_gen > self.epochs_to_pretrain:
            os.system(f'rm spec_gen_saves/just_last_{self.last_epoch_spec_gen - 1}.ckpt')
            
        self.last_epoch_spec_gen += 1
        
    def training_epoch_end(self, loss):
        
        self.last_epoch_asr += 1
        
        self.cuda()
        
        if self.last_epoch_asr == self.epochs_to_pretrain:
            self.train_spec_gen()
        elif self.last_epoch_asr >= self.epochs_to_pretrain:
            self.predict_text()
            self.train_spec_gen()
            self.predict_audio()
                  
        self.cuda()
                
        return super().training_epoch_end(loss)
    
    

In [9]:
with open(root_dir + 'conf/carnelinet/carnelinet_384.yaml') as f:
     asr_cfg = yaml.load(f)

In [10]:
model = SpeechChainModel.from_config_dict(DictConfig(asr_cfg['model']))

[NeMo I 2022-05-26 06:52:35 mixins:165] Tokenizer SentencePieceTokenizer initialized with 339 tokens
[NeMo I 2022-05-26 06:52:35 ctc_bpe_models:246] 
    Replacing placeholder number of classes (-1) with actual number of classes - 339
[NeMo I 2022-05-26 06:52:35 collections:186] Dataset loaded with 800 files totalling 0.62 hours
[NeMo I 2022-05-26 06:52:35 collections:187] 0 files were filtered totalling 0.00 hours
[NeMo I 2022-05-26 06:52:36 collections:186] Dataset loaded with 100 files totalling 0.08 hours
[NeMo I 2022-05-26 06:52:36 collections:187] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-05-26 06:52:36 ctc_bpe_models:309] Could not load dataset as `manifest_filepath` was None. Provided config : {'manifest_filepath': None, 'sample_rate': 22050, 'batch_size': 32, 'shuffle': False, 'use_start_end_token': False}


[NeMo I 2022-05-26 06:52:36 features:259] PADDING: 16
[NeMo I 2022-05-26 06:52:36 features:276] STFT using torch
[NeMo I 2022-05-26 06:52:36 tokenize_and_classify:88] Creating ClassifyFst grammars.


[NeMo W 2022-05-26 06:52:40 g2ps:84] apply_to_oov_word=None, it means that some of words will remain unchanged if they are not handled by one of rule in self.parse_one_word(). It is useful when you use tokenizer with set of phonemes and chars together, otherwise it can be not.


[NeMo I 2022-05-26 06:52:40 data:173] Loading dataset from /media/boris/F/Research_final/data/an4/paired.json.


...

[NeMo I 2022-05-26 06:55:21 data:207] Loaded dataset with 100 files.
[NeMo I 2022-05-26 06:55:21 data:209] Dataset contains 0.08 hours.
[NeMo I 2022-05-26 06:55:21 data:297] Pruned 0 files. Final dataset contains 100 files
[NeMo I 2022-05-26 06:55:21 data:299] Pruned 0.00 hours. Final dataset contains 0.08 hours.


[NeMo I 2022-05-26 06:55:21 features:259] PADDING: 1
[NeMo I 2022-05-26 06:55:21 features:276] STFT using torch
[NeMo I 2022-05-26 06:55:21 cloud:56] Found existing object /home/boris/.cache/torch/NeMo/NeMo_1.8.2/tts_melgan/38f156f172595e60f02169891e303590/tts_melgan.nemo.
[NeMo I 2022-05-26 06:55:21 cloud:62] Re-using file from: /home/boris/.cache/torch/NeMo/NeMo_1.8.2/tts_melgan/38f156f172595e60f02169891e303590/tts_melgan.nemo
[NeMo I 2022-05-26 06:55:21 common:747] Instantiating model from pre-trained checkpoint


[NeMo W 2022-05-26 06:55:21 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.AudioDataset
      manifest_filepath: /raid/LJSpeech/nvidia_ljspeech_train.json
      max_duration: null
      min_duration: 0.75
      n_segments: 16384
      trim: false
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-05-26 06:55:21 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.AudioDataset
      manifest_filepath: /raid/LJSpeech/nvidia_ljspeech_val.jso

[NeMo I 2022-05-26 06:55:21 features:259] PADDING: 0
[NeMo I 2022-05-26 06:55:21 features:276] STFT using torch
[NeMo I 2022-05-26 06:55:21 features:278] STFT using exact pad
[NeMo I 2022-05-26 06:55:23 save_restore_connector:209] Model MelGanModel was successfully restored from /home/boris/.cache/torch/NeMo/NeMo_1.8.2/tts_melgan/38f156f172595e60f02169891e303590/tts_melgan.nemo.


mkdir: cannot create directory ‘tmp’: File exists


In [ ]:
trainer = pl.Trainer(devices=1, 
                     accelerator='gpu',
                     max_epochs=300,
                     check_val_every_n_epoch=1, 
                     num_sanity_val_steps=0,
                     num_nodes=1,
                     accumulate_grad_batches=1,
#                      precision=16
                    )
    
    
trainer.fit(model)